In [1]:
import sys
import pickle
import numpy as np
import torch 

sys.path.append("../src")

import constants as constants
constants.load("../data/moves_10", True)

import neural_net
from neural_net import NeuralNet

""

Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners
Number of moves: 6233
Board size: 10


''

In [8]:
model = NeuralNet()

In [13]:
for _ in range(5000):
    occupancies = np.zeros((4, constants.BOARD_SIZE, constants.BOARD_SIZE), dtype=bool)
    neural_net.evaluate(model, occupancies)